In [2]:
import qutip as qt
import numpy as np
import matplotlib.pyplot as plt
from qutip.ipynbtools import version_table
from tqdm.notebook import tqdm
from matplotlib.animation import FuncAnimation
from scipy.linalg import expm
import pandas as pd

resol = 200
pi = np.pi
version_table()

Software,Version
QuTiP,5.1.1
Numpy,1.26.2
SciPy,1.11.3
matplotlib,3.7.3
Number of CPUs,11
BLAS Info,Generic
IPython,8.17.2
Python,"3.12.0 (v3.12.0:0fb18b02c8, Oct 2 2023, 09:45:56) [Clang 13.0.0 (clang-1300.0.29.30)]"
OS,posix [darwin]
Cython,3.0.11


<h1> Perform time evolution </h1>

In [10]:
def bs_evolution(L, T, dt, omega_0, omega_A, n_A, omega_max, g_0, sigma, x_0, delta_x, use_WW_approx = True, print_nb_modes = False):
    
    #quantization of the modes by keeping only a portion of them
    #omega_tab_p = np.linspace(0, omega_max , nb_omega)
    omega_tab_p = np.array([2*pi*n/L for n in range(1000000) if 2*pi*n/L <= omega_max])
    
    k_tab = np.sort(np.unique(np.concatenate((-omega_tab_p, omega_tab_p))))
    #keep only a portion of the modes 
    omega_tab = np.abs(k_tab)
    n_modes = len(k_tab)
    cutoff_test = np.inf

    if print_nb_modes:
        print("Number of modes: ", n_modes)
    
    #create the Hamiltonian
    dim_subspace = n_modes * (2*n_modes + 1) + 2*n_modes*n_A + n_A*(n_A-1)//2
    if print_nb_modes:
        print("Dimension of the subspace: ", dim_subspace)

    
    print("Generating the Hamiltonian...")
    #tab with the coupling parameters
    g_tab = np.zeros((n_modes, n_A), dtype=complex)
    for i in range(n_modes):
        for j in range(n_A):
            if use_WW_approx:
                g_tab[i, j] = g_0*np.sqrt(omega_A/L)*1j
            else:
                g_tab[i, j] = g_0*np.sqrt(omega_tab[i]/L)*1j #in my case, g is independant of the atom index


    #now define the Hamiltonian
    H_matrix = np.zeros((dim_subspace, dim_subspace), dtype=complex)
    index = 0 #dummy way to do it but I'm struggling to find a direct mapping

    ###Diagonal elements

    #diagonal elements with two photons in the horizontal direction
    for i in range(n_modes):
        for j in range(i, n_modes):
            H_matrix[index, index] = omega_tab[i] + omega_tab[j]
            index += 1

    #diagonal elements with two photons in the vertical direction
    for i in range(n_modes):
        for j in range(i, n_modes):
            H_matrix[index, index] = omega_tab[i] + omega_tab[j]
            index += 1
            
    #diagonal elements with two photons in distinct directions
    for i in range(n_modes**2):
        H_matrix[index,index] = omega_tab[int(i//n_modes)] + omega_tab[int(i%n_modes)]
        index += 1

    #diagonal elements with one horizontal photon and one excited atom
    for i in range(n_modes):
        for j in range(n_A):
            H_matrix[index,index] = omega_tab[i] + omega_A
            index += 1

    #diagonal elements with one vertical photon and the atom
    for i in range(n_modes):
        for j in range(n_A):
            H_matrix[index,index] = omega_tab[i] + omega_A
            index += 1

    #diagonal elements with two excited atoms
    if n_A > 1:
        for i in range(n_A):
            for j in range(i+1, n_A):
                H_matrix[index,index] = 2*omega_A
                index += 1

    ## Off diagonal elements

    #|2,0,0> <-> |1,0,1>
    dummy_index_0 = 0
    for i in range(n_modes):
        for j in range(i, n_modes):
            if omega_tab[i] + omega_tab[j] <= cutoff_test:
                dummy_index_1 = n_modes*(2*n_modes + 1)
                for k in range(n_modes):
                    for l in range(n_A):
                        if k == i:
                            if i == j:
                                H_matrix[dummy_index_1, dummy_index_0] = np.sqrt(2)*np.conjugate(g_tab[j,l])
                                H_matrix[dummy_index_0, dummy_index_1] = np.conjugate(H_matrix[dummy_index_1, dummy_index_0])
                            else:
                                H_matrix[dummy_index_1, dummy_index_0] = np.conjugate(g_tab[j,l])
                                H_matrix[dummy_index_0, dummy_index_1] = np.conjugate(H_matrix[dummy_index_1, dummy_index_0])
                        if k == j:
                            if i == j:
                                H_matrix[dummy_index_1, dummy_index_0] = np.sqrt(2)*np.conjugate(g_tab[i,l])
                                H_matrix[dummy_index_0, dummy_index_1] = np.conjugate(H_matrix[dummy_index_1, dummy_index_0])
                            else:
                                H_matrix[dummy_index_1, dummy_index_0] = np.conjugate(g_tab[i,l])
                                H_matrix[dummy_index_0, dummy_index_1] = np.conjugate(H_matrix[dummy_index_1, dummy_index_0])

                        dummy_index_1 += 1
                dummy_index_0 += 1

    #|0,2,0> <-> |0,1,1>
    dummy_index_0 = n_modes*(n_modes + 1)//2
    for i in range(n_modes):
        for j in range(i, n_modes):
            if omega_tab[i] + omega_tab[j] <= cutoff_test:
                dummy_index_1 = n_modes*(2*n_modes + 1) + n_modes*n_A
                for k in range(n_modes):
                    for l in range(n_A):
                        if k == i:
                            if i == j:
                                H_matrix[dummy_index_1, dummy_index_0] = np.sqrt(2)*np.conjugate(g_tab[j,l])
                                H_matrix[dummy_index_0, dummy_index_1] = np.conjugate(H_matrix[dummy_index_1, dummy_index_0])
                            else:
                                H_matrix[dummy_index_1, dummy_index_0] = np.conjugate(g_tab[j,l])
                                H_matrix[dummy_index_0, dummy_index_1] = np.conjugate(H_matrix[dummy_index_1, dummy_index_0])
                        if k == j:
                            if i == j:
                                H_matrix[dummy_index_1, dummy_index_0] = np.sqrt(2)*np.conjugate(g_tab[i,l])
                                H_matrix[dummy_index_0, dummy_index_1] = np.conjugate(H_matrix[dummy_index_1, dummy_index_0])
                            else:
                                H_matrix[dummy_index_1, dummy_index_0] = np.conjugate(g_tab[i,l])
                                H_matrix[dummy_index_0, dummy_index_1] = np.conjugate(H_matrix[dummy_index_1, dummy_index_0])

                        dummy_index_1 += 1
                dummy_index_0 += 1

    #|1,1,0> <-> |1,0,1>
    dummy_index_0 = n_modes*(n_modes + 1)
    for i in range(n_modes):
        for j in range(n_modes):
                if omega_tab[i] + omega_tab[j] <= cutoff_test:
                    dummy_index_1 = n_modes*(2*n_modes + 1) 
                    for k in range(n_modes):
                        for l in range(n_A):
                            if k == i:
                                H_matrix[dummy_index_1, dummy_index_0] = np.conjugate(g_tab[j,l])
                                H_matrix[dummy_index_0, dummy_index_1] = np.conjugate(H_matrix[dummy_index_1, dummy_index_0])
                            if k == j:
                                H_matrix[dummy_index_1, dummy_index_0] = np.conjugate(g_tab[i,l])
                                H_matrix[dummy_index_0, dummy_index_1] = np.conjugate(H_matrix[dummy_index_1, dummy_index_0])
                                
                            dummy_index_1 += 1
                    dummy_index_0 += 1
        
    #|1,1,0> <-> |0,1,1>
    dummy_index_0 = n_modes*(n_modes + 1)
    for i in range(n_modes):
        for j in range(n_modes):
                if omega_tab[i] + omega_tab[j] <= cutoff_test:
                    dummy_index_1 = n_modes*(2*n_modes + 1) + n_modes*n_A
                    for k in range(n_modes):
                        for l in range(n_A):
                            if k == i:
                                H_matrix[dummy_index_1, dummy_index_0] = np.conjugate(g_tab[j,l])
                                H_matrix[dummy_index_0, dummy_index_1] = np.conjugate(H_matrix[dummy_index_1, dummy_index_0])
                            if k == j:
                                H_matrix[dummy_index_1, dummy_index_0] = np.conjugate(g_tab[i,l])
                                H_matrix[dummy_index_0, dummy_index_1] = np.conjugate(H_matrix[dummy_index_1, dummy_index_0])
                                
                            dummy_index_1 += 1
                    dummy_index_0 += 1


    if n_A > 1:
        #|1,0,1> <-> |0,0,2>
        dummy_index_0 = n_modes*(2*n_modes + 1)
        for i in range(n_modes):
            if omega_tab[i] <= cutoff_test:
                for j in range(n_A):
                    dummy_index_1 = n_modes*(2*n_modes + 1) + 2*n_modes*n_A
                    for k in range(n_A):
                        for l in range(k+1, n_A):                            
                            if k == j:
                                H_matrix[dummy_index_1, dummy_index_0] = np.conjugate(g_tab[i,l])
                                H_matrix[dummy_index_0, dummy_index_1] = np.conjugate(H_matrix[dummy_index_1, dummy_index_0])
                            if l == j:
                                H_matrix[dummy_index_1, dummy_index_0] = np.conjugate(g_tab[i,k])
                                H_matrix[dummy_index_0, dummy_index_1] = np.conjugate(H_matrix[dummy_index_1, dummy_index_0])
                            #print("ok")
                            dummy_index_1 += 1
                    dummy_index_0 += 1
                
        #|0,1,1> <-> |0,0,2>
        dummy_index_0 = n_modes*(2*n_modes + 1) + n_modes*n_A
        for i in range(n_modes):
            if omega_tab[i] <= cutoff_test:
                for j in range(n_A):
                    dummy_index_1 = n_modes*(2*n_modes + 1) + 2*n_modes*n_A
                    for k in range(n_A):
                        for l in range(k+1, n_A):
                            if k == j:
                                H_matrix[dummy_index_1, dummy_index_0] = np.conjugate(g_tab[i,k])
                                H_matrix[dummy_index_0, dummy_index_1] = np.conjugate(H_matrix[dummy_index_1, dummy_index_0])
                            if l == j:
                                H_matrix[dummy_index_1, dummy_index_0] = np.conjugate(g_tab[i,l])
                                H_matrix[dummy_index_0, dummy_index_1] = np.conjugate(H_matrix[dummy_index_1, dummy_index_0])
                            #print("ok")
                            dummy_index_1 += 1
                    dummy_index_0 += 1

    H = qt.Qobj(H_matrix)

    #hermicity check
    if not H.isherm:
        print("Warning: the Hamiltonian is not hermitian")
        return None
    

    #also get the free Hamiltonian
    H_free = qt.Qobj(np.diag(np.diag(H_matrix)))

    print("Preparing the initial state...")
    ##Now prepare the initial state
    k_bar = k_tab[np.argmin(np.abs(k_tab - omega_0))]
    x_0_a = x_0
    x_0_b = x_0 + delta_x

    init_state = 0
    for i in range(n_modes**2):
        k_a = k_tab[int(i//n_modes)]
        k_b = k_tab[int(i%n_modes)]
        if k_a > 0 and k_b > 0:
            c_a = np.exp(-0.5*sigma**2*(k_a - k_bar)**2 -1j * k_a * x_0_a)
            c_b = np.exp(-0.5*sigma**2*(k_b - k_bar)**2 -1j * k_b * x_0_b)
            init_state += c_a * c_b * qt.basis(dim_subspace, n_modes*(n_modes+1) + i)

    init_state = init_state.unit()


    print("Conducting time evolution...")
    ##Conduct time evolution
    n_step = int(T/dt)
    times = np.linspace(0, T, n_step)
    result = qt.sesolve(H, init_state, times, options={"nsteps":10000, "progress_bar": True, "store_states": True})
    state_list = result.states

    print("Conducting free evolution...")
    #also get the free evolution of the initial state
    result_free = qt.sesolve(H_free, init_state, times, options={"nsteps":10000, "store_states": True, "progress_bar": True})
    state_list_free = result_free.states
    print("Done.")

    final_overlap_free = np.abs(state_list_free[-1].dag() * state_list[-1])**2

    return final_overlap_free


In [7]:
L = 10*pi
T = L/2
dt = 0.1

#Jaynes-Cummings parameter g_0 and energies 
g_0 = 3
omega_A = 3
n_A = 1
gamma = 2*g_0**2*omega_A

#initial photon wavepacket parameters
sigma = L/16
sigma_momentum = 1/sigma
x_0 = -L/4
omega_0 = omega_A - gamma/2
omega_0 = 1
delta_x = L/8

#numerical implementation of momentum space
omega_max = 10
nb_omega = 32

print("Coupling g(k,j): ", g_0*np.sqrt(omega_A / L))
print("sigma_momentum: ", sigma_momentum)
print("Decay rate gamma: ", gamma)


Coupling g(k,j):  0.9270580848556551
sigma_momentum:  0.5092958178940651
Decay rate gamma:  54


In [8]:
final_overlap_free = bs_evolution(L, T, dt, omega_0, nb_omega, omega_A, n_A, omega_max, g_0, sigma, x_0, delta_x, use_WW_approx=True, print_nb_modes=True)

Number of modes:  101
Dimension of the subspace:  20705
Generating the Hamiltonian...
Preparing the initial state...
Conducting time evolution...
10.3%. Run time:  18.61s. Est. time left: 00:00:02:42
20.5%. Run time:  38.49s. Est. time left: 00:00:02:29
30.1%. Run time:  54.90s. Est. time left: 00:00:02:07
40.4%. Run time:  70.95s. Est. time left: 00:00:01:44
50.0%. Run time:  85.96s. Est. time left: 00:00:01:25
60.3%. Run time: 102.05s. Est. time left: 00:00:01:07
70.5%. Run time: 118.42s. Est. time left: 00:00:00:49
80.1%. Run time: 133.75s. Est. time left: 00:00:00:33
90.4%. Run time: 149.90s. Est. time left: 00:00:00:15
100.0%. Run time: 164.99s. Est. time left: 00:00:00:00
Total run time: 164.99s
Conducting free evolution...
10.3%. Run time:   4.29s. Est. time left: 00:00:00:37
20.5%. Run time:   6.78s. Est. time left: 00:00:00:26
30.1%. Run time:   9.27s. Est. time left: 00:00:00:21
40.4%. Run time:  12.13s. Est. time left: 00:00:00:17
50.0%. Run time:  14.85s. Est. time left: 00

In [9]:
print("Final overlap with the free evolution: ", final_overlap_free)
print("Theory value: ", 0.5*(1- np.exp(- (delta_x**2) / (2*sigma**2))))

Final overlap with the free evolution:  0.3725456925027646
Theory value:  0.43233235838169365


<h1> Reproduce a HOM effect </h1>

In [ ]:
L = 10*pi
T = L/2
dt = 0.1

#Jaynes-Cummings parameter g_0 and energies 
g_0 = 3
omega_A = 3
n_A = 1

#initial photon wavepacket parameters
sigma = L/16
sigma_momentum = 1/sigma
x_0 = -L/4
omega_0 = 1

#numerical implementation of momentum space
omega_max = 10
nb_omega = 32

nb_pts_x = 10
delta_x_list = np.linspace(-2*sigma, 2*sigma, nb_pts_x)
final_overlap_free_list = np.zeros(nb_pts_x)

for i in range(nb_pts_x):
    delta_x = delta_x_list[i]
    final_overlap_free_list[i] = bs_evolution(L, T, dt, omega_0, omega_A, n_A, omega_max, g_0, sigma, x_0, delta_x, use_WW_approx=True, print_nb_modes=False)
    #print("Final overlap with the free evolution (delta_x = ", delta_x, "): ", final_overlap_free_list[i])
    #xprint("Theory value: ", 0.5*(1- np.exp(- (delta_x**2) / (2*sigma**2))))